# Compare cooltools expected counts and HiCONA normalization factors
**Note**: Not using directly hicona functions to filter counts and compute normalization factors since they are not meant to be in the final public API and therefore are susceptible to changes, which could thus break the script functionality.

In [1]:
import time

from cooler.util import open_hdf5
import cooltools
import matplotlib.pyplot as plt
from numpy import log2, nan
import pandas as pd
import seaborn as sns

import hicona

In [2]:
def get_chrom_view(cool_obj, chrom_name):
    """Get chromosome view for cooltools."""

    view = pd.DataFrame(
        data = [[chrom_name, 0, cool_obj.chromsizes[chrom_name], chrom_name]], 
        columns=["chrom", "start", "end", "name"])

    return view


In [3]:
def get_cooltools_counts(cool_obj, region, max_bin_diff, do_smooth=False, do_time = False):
    """Get chromsome expected counts according to cooltools."""
    
    start_cooltools = time.time()
    cvd = cooltools.expected_cis(
        clr=cool_obj, 
        view_df= region,
        smooth=do_smooth, 
        aggregate_smoothed=do_smooth, 
        nproc=1
    )
    cooltools_time = time.time() - start_cooltools
    ind = cvd[cvd["dist"] > max_bin_diff].index
    cvd.drop(ind, inplace=True)
    
    if do_time:
        print(f"Cooltools, smooth = {do_smooth}, took {cooltools_time} seconds.")
    
    return cvd


In [4]:
def get_hicona_counts(cool_obj, chrom_name, max_bin_diff, do_time = False):
    """Get chromosome normalized counts according to HiCONA."""
    
    start_hicona = time.time()
    
    lower, upper = cool_obj.extent(chrom_name)
    with open_hdf5(cool_obj.store, mode="r") as h5_handle:
        h5_grp = h5_handle[cool_obj.root]
        lo_pix = h5_grp["indexes/bin1_offset"][lower]
        hi_pix = h5_grp["indexes/bin1_offset"][upper]
    
    pix_df = cool_obj.pixels()[lo_pix:hi_pix]
    indexer = pix_df[
        (pix_df["bin2_id"] - pix_df["bin1_id"] > max_bin_diff)
        | (pix_df["bin1_id"] == pix_df["bin2_id"])
        | (pix_df["bin2_id"] >= upper)
    ].index
    pix_df.drop(indexer, inplace=True)
    
    start_size = pix_df.shape[0]
    pix_df.drop_duplicates(subset=["bin1_id", "bin2_id"], inplace=True)
    size_diff = start_size - pix_df.shape[0]
    if size_diff != 0:
        print(f"Warning, {size_diff} duplicate rows were dropped.")

    pix_df["bin_difference"] = pix_df["bin2_id"] - pix_df["bin1_id"]
    group_counts = pix_df.groupby("bin_difference")["count"]
    decay_curve = group_counts.agg("median").to_frame()
    
    if do_time:
        hicona_time = time.time() - start_hicona
    
    return decay_curve


In [5]:
def merge_frames(cool_df, tools_coarse):
    """Merge expected counts obtained through different methods for plotting purposes."""
    
    _COLS_TO_DROP = ["region1", "region2", "dist", "n_valid", "count.sum", "balanced.sum", "balanced.avg"]
    _VALUE_VARS = ["hicona", "cooltools"]
        
    merged = tools_coarse.copy()
    merged.drop(_COLS_TO_DROP, axis=1, inplace = True)
    merged = merged.merge(cool_df, how="outer", left_index=True, right_index=True)
    merged.rename(columns={"count.avg": "cooltools", "count":"hicona"}, inplace=True)
    
    # Cooltools does not provide an estimate for distance 0 and 1 (too noisy)
    merged["hicona"][0] = nan
    merged["hicona"][1] = nan
        
    merged.reset_index(inplace=True, names=["bin_difference"])
    merged = pd.melt(merged, id_vars=["bin_difference"], value_vars=_VALUE_VARS)
    merged.rename(columns={"variable":"method", "value":"exp_count"}, inplace=True)
    
    return merged


In [6]:
def lin_plot(merged):
    """Plot with y axis in linear scale."""
    
    sns.set_style(style="ticks")
    plt.rcParams.update({'font.size': 12})
    max_pos = merged[(merged["method"]=="cooltools") & (merged["bin_difference"] == res * 2)].iloc[0]["exp_count"]
    max_tick = int(max_pos // 2 * 2)  # Round to closest smaller even number

    fig, axes = plt.subplots(1, 1, figsize=(6,6))
    fig.suptitle(f"{chrom}, res: {res:,} bp, file: {F_NAME}", fontsize=20, fontweight=10)
    fig.tight_layout(rect=[0, 0, 1, 1])

    sns.scatterplot(
        data=merged, 
        x='bin_difference', 
        y='exp_count', 
        hue='method', 
        s=10, 
        linewidth=0.01, 
        ax=axes,
        palette="tab10"
    )
    axes.set_xlabel("genomic distance (bp)", fontsize=12)
    axes.set_ylabel("counts statistic", fontsize=12)
    axes.set(xscale="log", yscale="log")
    axes.legend(title="Method", fontsize=12, title_fontsize=12)
    axes.axhline(1, color="grey", linestyle="--", zorder=0)
    axes.tick_params(axis='both', labelsize=10)
    axes.spines[['right', 'top']].set_visible(False)
    axes.set_xlabel("genomic distance (bp)", fontsize=12)

    plt.savefig(OUT_FOLDER + f"/{chrom}_{res}_{F_NAME}_lin.png", dpi=600, bbox_inches='tight')
    plt.close()


In [7]:
def log_plot(merged):
    """Plot with y axis in log scale."""
    
    sns.set_style(style="ticks")
    plt.rcParams.update({'font.size': 12})
    max_pos = merged[(merged["method"]=="cooltools") & (merged["bin_difference"] == res * 2)].iloc[0]["exp_count"]
    max_tick = int(max_pos // 2 * 2)  # Round to closest smaller even number

    fig, axes = plt.subplots(1, 1, figsize=(6,6))
    fig.suptitle(f"{chrom}, res: {res:,} bp, file: {F_NAME}", fontsize=20, fontweight=10)
    fig.tight_layout(rect=[0, 0, 1, 1])

    sns.scatterplot(
        data=merged, 
        x='bin_difference', 
        y='exp_count', 
        hue='method', 
        s=10, 
        linewidth=0.01, 
        ax=axes,
        palette="tab10"
    )
    axes.set_xlabel("genomic distance (bp)", fontsize=12)
    axes.set_ylabel("counts statistic", fontsize=12)
    axes.set(xscale="log", yscale="linear")
    axes.legend(title="Method", fontsize=12, title_fontsize=12)
    axes.axhline(1, color="grey", linestyle="--", zorder=0)
    axes.tick_params(axis='both', labelsize=10)
    axes.spines[['right', 'top']].set_visible(False)
    axes.set_xlabel("genomic distance (bp)", fontsize=12)
    # axes.set_yticks([n for n in range(0,max_tick+1,2)] + [1])
        
    plt.savefig(OUT_FOLDER + f"/{chrom}_{res}_{F_NAME}_log.png", dpi=600, bbox_inches='tight')
    plt.close()


In [8]:
def both_plot(merged):
    """Plot both with y axis both in log and in linear scale, side by side."""
    
    sns.set_style(style="ticks")
    plt.rcParams.update({'font.size': 16})
    max_pos = merged[(merged["method"]=="cooltools") & (merged["bin_difference"] == res * 2)].iloc[0]["exp_count"]
    max_tick = int(max_pos // 2 * 2)  # Round to closest smaller even number

    fig, axes = plt.subplots(1, 2, figsize=(12,6))
    fig.suptitle(f"{chrom}, res: {res:,} bp, file: {F_NAME}", fontsize=20, fontweight=10)
    fig.tight_layout(rect=[0, 0, 1, 1])

    sns.scatterplot(
        data=merged, 
        x='bin_difference', 
        y='exp_count', 
        hue='method', 
        s=10, 
        linewidth=0.01, 
        ax=axes[0],
        palette="tab10"
    )
    axes[0].set_xlabel("genomic distance (bp)", fontsize=16)
    axes[0].set_ylabel("counts statistic", fontsize=16)
    axes[0].set(xscale="log", yscale="log")
    axes[0].get_legend().remove()
    axes[0].axhline(1, color="grey", linestyle="--", zorder=0)
    axes[0].tick_params(axis='both', labelsize=12)
    axes[0].spines[['right', 'top']].set_visible(False)

    sns.scatterplot(
        data=merged, 
        x='bin_difference', 
        y='exp_count', 
        hue='method', 
        s=10, 
        linewidth=0.01, 
        ax=axes[1],
        palette="tab10"
    )
    axes[1].set_xlabel("genomic distance (bp)", fontsize=16)
    axes[1].set_ylabel(None)
    axes[1].legend(title="Method", fontsize=16, title_fontsize=16)
    axes[1].set(xscale="log", yscale="linear")
    axes[1].axhline(1, color="grey", linestyle="--", zorder=0)
    axes[1].tick_params(axis='both', labelsize=12)
    # axes[1].set_yticks([n for n in range(0,max_tick+1,2)] + [1])
    axes[1].spines[['right', 'top']].set_visible(False)

    plt.savefig(OUT_FOLDER + f"/{chrom}_{res}_{F_NAME}_both.png", dpi=600, bbox_inches='tight')
    plt.close()


In [9]:
FILE_PATH = "test_files/4DNFI8ZYY7VT_dekker.mcool::resolutions/10000" # 4DNucleome id
OUT_FOLDER = "plots/hicona_vs_cooltools"
F_NAME = "HFFc6 7.3 gb"
MAX_GEN_DIST = 200_000_000

cool_handle = hicona.HiconaCooler(FILE_PATH)
res = cool_handle.info["bin-size"]
max_bin_diff = -(-MAX_GEN_DIST // res)

In [10]:
DONE = []
for chrom in cool_handle.chromnames:
    print(f"Starting to work on {chrom}")
    if chrom in DONE:
        print(f"{chrom} was already processed, skipping")
        continue
    view = get_chrom_view(cool_handle, chrom)
    tool = get_cooltools_counts(cool_handle, view, max_bin_diff)
    hico = get_hicona_counts(cool_handle, chrom, max_bin_diff)
    merged = merge_frames(hico, tool)
    merged["bin_difference"] *= res  # Convert to bp
    lin_plot(merged)
    log_plot(merged)
    both_plot(merged)

Starting to work on chr1
Starting to work on chr2
Starting to work on chr3
Warning, 1454 duplicate rows were dropped.
Starting to work on chr4
Warning, 1294 duplicate rows were dropped.
Starting to work on chr5
Starting to work on chr6
Starting to work on chr7
Starting to work on chr8
Starting to work on chr9
Starting to work on chr10
Warning, 758 duplicate rows were dropped.
Starting to work on chr11
Warning, 564 duplicate rows were dropped.
Starting to work on chr12
Warning, 587 duplicate rows were dropped.
Starting to work on chr13
Warning, 343 duplicate rows were dropped.
Starting to work on chr14
Starting to work on chr15
Warning, 397 duplicate rows were dropped.
Starting to work on chr16
Starting to work on chr17
Starting to work on chr18
Starting to work on chr19
Starting to work on chr20
Warning, 213 duplicate rows were dropped.
Starting to work on chr21
Starting to work on chr22
Starting to work on chrX
Starting to work on chrY
